# Optimización Matemática - QDA

## Consigna

**Sugerencia:** considerar combinaciones adecuadas de `transpose`, `reshape` y, ocasionalmente, `flatten`. Explorar la dimensionalidad de cada elemento antes de implementar las clases.

Debido a la forma cuadrática de QDA, no se puede predecir para *n* observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de *n x n* en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

1. Implementar el modelo `FasterQDA` (se recomienda heredarlo de TensorizedQDA) de manera de eliminar el ciclo for en el método predict.
2. Comparar los tiempos de predicción de `FasterQDA` con `TensorizedQDA` y `QDA`.
3. Mostrar (puede ser con un print) dónde aparece la mencionada matriz de *n x n*, donde *n* es la cantidad de observaciones a predecir.
4.Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de *n x n* usando matrices de *n x p*.
5.Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente. ¿Hay cambios en los tiempos de predicción?

## Modelo

### Imports

In [ ]:
import numpy as np
from numpy.linalg import det, inv

In [ ]:
### ClassEncoder

In [ ]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype

  def _map_reshape(self, f, arr):
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)

  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

### BaseBayesianClassifier

In [ ]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # first encode the classes
    y = self.encoder.fit_transform(y)

    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1] # nro. de observaciones (Columnas de X)
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt) # 1 x m

    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      # Es la predicción numérica 0, 1 ó 2

      y_hat[i] = self.encoder.names[encoded_y_hat_i]
      # Es el valor de la predicción en texto (descripción)

      # y_hat es el array de predicciones (descripciones) 90 x 1
      # Se devuelve 1 x 90

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # Log posteriori es un array de las probabilidades de cada clase
    # Ej.: [array([[-289.46784276]]), array([[2.3818009]]), array([[-2.34841277]])]
    # Devuelve el índice con la probabilidad mayor (0, 1, ó 2)

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

### QDA

In [ ]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):

    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):

    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

### TensorizedQDA

In [ ]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

### FasterQDA

In [ ]:
class FasterQDA(TensorizedQDA):

  def predict(self, X):
    # X es (n,n), n variables, m observaciones
    y_hat = np.empty(X.shape[1], dtype=self.encoder.fmt) # m x 1

    # unbiased es (3, n, m) (clases, variables, observaciones)
    unbiased_X = X - self.tensor_means

    # unbiased_X es (3, n, m)
    # unbiased_X.transpose es (3, m, n)
    # tensor_inv_cov.shape es (3, n, n)
    # log_a_priori.shape es (3,)

    # inner_prod es (3, m, m)
    inner_prod = unbiased_X.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_X

    # print("Resultado del producto interno (3, m, m)")
    # print(inner_prod)

    # tomar los elementos de la diagonal para reducir la matriz de mxm
    inner_prod_d = np.array([np.diagonal(inner_prod[i]) for i in range(inner_prod.shape[0])])

    m_a_priori = np.tile(self.log_a_priori, (X.shape[1], 1))
    m_log_det = np.tile(np.log(det(self.tensor_inv_cov)), (X.shape[1], 1))
    m_result = m_a_priori + (0.5*m_log_det - 0.5*inner_prod_d.T)
    encoded_y_hat = np.argmax(m_result, axis=1)
    # encoded_y_hat es (m,)

    y_hat[:len(encoded_y_hat)] = self.encoder.names[encoded_y_hat]

    # (1, m)
    return y_hat.reshape(1,-1)


### RealFasterQDA


Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de *n x n* usando matrices de *n x p*.

Sean:

**A** una matriz de *n x p*

$$
\mathbf{A} =
\begin{pmatrix}
a_{11} & ... & a_{1p} \\
a_{21} & ... & a_{2p} \\
... & ... & ... \\
a_{n1} & ... & a_{np} \\
\end{pmatrix}
$$

**B** una matriz de *p x n*

$$
\mathbf{B} =
\begin{pmatrix}
b_{11} & ... & b_{1n} \\
b_{21} & ... & b_{2n} \\
... & ... & ... \\
b_{p1} & ... & b_{pn} \\
\end{pmatrix}
$$


El producto matricial entre **A** y **B** resulta una matriz de *n x n* de la forma:

$$
\\
\begin{pmatrix}
a_{11}b_{11} + a_{12}b_{21} + ... + a_{1p}b_{p1} & ... & a_{11}b_{1n} + a_{12}b_{2n} + ... + a_{1p}b_{pn} \\
a_{21}b_{11} + a_{22}b_{21} + ... + a_{2p}b_{p1} & ... & a_{21}b_{1n} + a_{22}b_{2n} + ... + a_{2p}b_{pn} \\
... & ... & ... \\
a_{n1}b_{11} + a_{n2}b_{21} + ... + a_{np}b_{p1} & ... & a_{n1}b_{1n} + a_{n2}b_{2n} + ... + a_{np}b_{pn} \\
\end{pmatrix}
$$


La diagonal resulta un vector de *n* elementos de la forma:
$$
\\
\begin{pmatrix}
a_{11}b_{11} + a_{12}b_{21} + ... + a_{1p}b_{p1} \\
a_{21}b_{12} + a_{22}b_{22} + ... + a_{2p}b_{p2} \\
... \\
a_{n1}b_{1n} + a_{n2}b_{2n} + ... + a_{np}b_{pn} \\
\end{pmatrix}
$$


Por otro lado, **B** traspuesta es la matriz de *n x p* de la forma:

$$
\mathbf{B^T} =
\begin{pmatrix}
b_{11} & ... & b_{p1} \\
b_{12} & ... & b_{p2} \\
... & ... & ... \\
b_{1n} & ... & b_{pn} \\
\end{pmatrix}
$$

Al hacer el producto elemento a elemento con **A**, quedaría de la forma:


$$
\begin{pmatrix}
a_{11}b_{11} & ... & a_{1p}b_{p1} \\
a_{21}b_{12} & ... & a_{2p}b_{p2} \\
... & ... & ... \\
a_{n1}b_{1n} & ... & a_{np}b_{pn} \\
\end{pmatrix}
$$


Finalmente, si sumamos los elementos de las filas en una sola columna, resulta el mismo vector diagonal anterior, demostrándose la igualdad:


$$
\begin{pmatrix}
a_{11}b_{11} + a_{12}b_{21} + ... + a_{1p}b_{p1} \\
a_{21}b_{12} + a_{22}b_{22} + ... + a_{2p}b_{p2} \\
... \\
a_{n1}b_{1n} + a_{n2}b_{2n} + ... + a_{np}b_{pn} \\
\end{pmatrix}
$$









In [ ]:
class RealFasterQDA(TensorizedQDA):


  def predict(self, X):
    y_hat = np.empty(X.shape[1], dtype=self.encoder.fmt)

    unbiased_X = X - self.tensor_means

    #A => (unbiased_X.transpose(0,2,1) @ self.tensor_inv_cov)
    #B_t => (np.transpose(unbiased_X, (0,2,1)))
    # diag (A@B) = np.sum(A*B_t, axis=1)

    inner_prod_d = np.sum((unbiased_X.transpose(0,2,1) @ self.tensor_inv_cov)*(np.transpose(unbiased_X, (0,2,1))), axis=2)

    m_a_priori = np.tile(self.log_a_priori, (X.shape[1], 1))
    m_log_det = np.tile(np.log(det(self.tensor_inv_cov)), (X.shape[1], 1))
    m_result = m_a_priori + (0.5*m_log_det - 0.5*inner_prod_d.T)
    encoded_y_hat = np.argmax(m_result, axis=1)

    y_hat[:len(encoded_y_hat)] = self.encoder.names[encoded_y_hat]

    return y_hat.reshape(1,-1)

## Código para pruebas

### Hiperparámetros

In [ ]:
# hiperparámetros
rng_seed = 6543

### DataSets

In [ ]:
from sklearn.datasets import load_iris, fetch_openml

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

# showing for iris
X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

X: (150, 4), Y:(150, 1)


In [ ]:
# peek data matrix
X_full[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [ ]:
# peek target vector
y_full[:5]

array([['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa']], dtype='<U10')

### Split del DataSet en Train/Test

In [ ]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split

def split_transpose(X, y, test_sz, random_state):
    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=random_state)

    # transpose so observations are column vectors
    return X_train.T, y_train.T, X_test.T, y_test.T

def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, rng_seed)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4, 90) (1, 90) (4, 60) (1, 60)


### Entrenamiento y performance

In [ ]:
qda = QDA()

qda.fit(train_x, train_y)

In [ ]:
train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


In [ ]:
t_qda = TensorizedQDA()

t_qda.fit(train_x, train_y)

In [ ]:
ttrain_acc = accuracy(train_y, t_qda.predict(train_x))
ttest_acc = accuracy(test_y, t_qda.predict(test_x))
print(f"Train (apparent) error is {1-ttrain_acc:.4f} while test error is {1-ttest_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


In [ ]:
f_qda = FasterQDA()

f_qda.fit(train_x, train_y)

In [ ]:
ftrain_acc = accuracy(train_y, f_qda.predict(train_x))
ftest_acc = accuracy(test_y, f_qda.predict(test_x))
print(f"Train (apparent) error is {1-ftrain_acc:.4f} while test error is {1-ftest_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


In [ ]:
rf_qda = RealFasterQDA()

rf_qda.fit(train_x, train_y)

In [ ]:
rftrain_acc = accuracy(train_y, rf_qda.predict(train_x))
rftest_acc = accuracy(test_y, rf_qda.predict(test_x))
print(f"Train (apparent) error is {1-rftrain_acc:.4f} while test error is {1-rftest_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


In [ ]:
%%timeit

qda.predict(test_x)

6.22 ms ± 652 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit

t_qda.predict(test_x)

1.82 ms ± 262 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit

f_qda.predict(test_x)

103 µs ± 24.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%%timeit

rf_qda.predict(test_x)

95.4 µs ± 20.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%%timeit

model = QDA()
model.fit(train_x, train_y)
model.predict(test_x)

21.2 ms ± 7.36 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
